In [1]:
#import libraries
import pandas as pd

In [3]:
df=pd.read_csv(r"C:\Users\raksh\Downloads\archive (1)\test.csv")

## Data Loading
The dataset was loaded using pandas from a CSV file. The shape of the dataset was checked to understand how many rows and columns are present before starting the cleaning process.


In [7]:
df.info

<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0     1461          20       RH         80.0    11622   Pave   NaN      Reg   
1     1462          20       RL         81.0    14267   Pave   NaN      IR1   
2     1463          60       RL         74.0    13830   Pave   NaN      IR1   
3     1464          60       RL         78.0     9978   Pave   NaN      IR1   
4     1465         120       RL         43.0     5005   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Ut

## Initial Data Exploration
The dataset was explored using head() and info() to understand the structure, column names, data types, and to check which columns contain missing values.


In [8]:
#Check missing values
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
MasVnrType       894
FireplaceQu      730
LotFrontage      227
GarageYrBlt       78
GarageQual        78
GarageFinish      78
GarageCond        78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrArea        15
MSZoning           4
BsmtHalfBath       2
Utilities          2
Functional         2
BsmtFullBath       2
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
KitchenQual        1
TotalBsmtSF        1
Exterior2nd        1
GarageCars         1
Exterior1st        1
GarageArea         1
SaleType           1
dtype: int64

## Missing Value Analysis
Missing values were identified using isnull().sum(). This helped in deciding which columns need to be cleaned and what method should be used to handle them.


In [9]:
#Handling missing values
df_clean = df.copy()

# Separate numerical and categorical columns
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df_clean.select_dtypes(include=["object"]).columns

# Fill numeric columns with median
for col in num_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Fill categorical columns with mode
for col in cat_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Check again
df_clean.isnull().sum().sum()


0

## Handling Missing Values
For numerical columns, missing values were filled using the median to avoid the influence of outliers. For categorical columns, missing values were filled using the mode to preserve the most common category.


In [10]:
#Removing duplicates

before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after = df_clean.shape[0]

print("Rows before:", before)
print("Rows after:", after)


Rows before: 1459
Rows after: 1459


In [11]:
#Datatype conversion
df_clean["YrSold"] = df_clean["YrSold"].astype(int)


## Data Type Correction
Important columns such as year-related fields were converted to appropriate numeric data types so that calculations and feature engineering can be performed correctly.


In [14]:
#Feature Engineering

# Age of the house
if "YrSold" in df_clean.columns and "YearBuilt" in df_clean.columns:
    df_clean["HouseAge"] = df_clean["YrSold"] - df_clean["YearBuilt"]

# Total square feet
if "TotalBsmtSF" in df_clean.columns and "GrLivArea" in df_clean.columns:
    df_clean["TotalSF"] = df_clean["TotalBsmtSF"] + df_clean["GrLivArea"]

# Price per square feet (ONLY if SalePrice exists)
if "SalePrice" in df_clean.columns:
    df_clean["PricePerSF"] = df_clean["SalePrice"] / df_clean["GrLivArea"]

df_clean.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HouseAge,TotalSF
0,1461,20,RH,80.0,11622,Pave,Grvl,Reg,Lvl,AllPub,...,Ex,MnPrv,Shed,0,6,2010,WD,Normal,49,1778.0
1,1462,20,RL,81.0,14267,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Gar2,12500,6,2010,WD,Normal,52,2658.0
2,1463,60,RL,74.0,13830,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Shed,0,3,2010,WD,Normal,13,2557.0
3,1464,60,RL,78.0,9978,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Shed,0,6,2010,WD,Normal,12,2530.0
4,1465,120,RL,43.0,5005,Pave,Grvl,IR1,HLS,AllPub,...,Ex,MnPrv,Shed,0,1,2010,WD,Normal,18,2560.0


## Feature Engineering
New features such as HouseAge and TotalSF were created from existing columns to provide better information for analysis and machine learning models. These features help the model understand the data more effectively.


In [19]:
#Save cleaned dataset
df_clean.to_csv("cleaned.csv", index=False)

print("Cleaned file saved as cleaned.csv")


Cleaned file saved as cleaned.csv


## Saving the Cleaned Dataset
After completing all cleaning and transformation steps, the final dataset was saved as a new CSV file so it can be directly used for further analysis and model building.


In [16]:
df_clean.shape
df_clean.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,HouseAge,TotalSF
0,1461,20,RH,80.0,11622,Pave,Grvl,Reg,Lvl,AllPub,...,Ex,MnPrv,Shed,0,6,2010,WD,Normal,49,1778.0
1,1462,20,RL,81.0,14267,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Gar2,12500,6,2010,WD,Normal,52,2658.0
2,1463,60,RL,74.0,13830,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Shed,0,3,2010,WD,Normal,13,2557.0
3,1464,60,RL,78.0,9978,Pave,Grvl,IR1,Lvl,AllPub,...,Ex,MnPrv,Shed,0,6,2010,WD,Normal,12,2530.0
4,1465,120,RL,43.0,5005,Pave,Grvl,IR1,HLS,AllPub,...,Ex,MnPrv,Shed,0,1,2010,WD,Normal,18,2560.0
